# ColBERTv2: Indexing & Search Notebook

We recommend selecting "GPU" as your hardware accelerator in the runtime settings.

First, we'll import the relevant classes. Note that `Indexer` and `Searcher` are the key actors here.

In [1]:
!git -C ColBERT/ pull || git clone https://github.com/hichewss/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')


fatal: cannot change to 'ColBERT/': No such file or directory
Cloning into 'ColBERT'...
remote: Enumerating objects: 1346, done.
remote: Counting objects: 100% (727/727), done.
remote: Compressing objects: 100% (325/325), done.
remote: Total 1346 (delta 541), reused 402 (delta 402), pack-reused 619
Receiving objects: 100% (1346/1346), 816.60 KiB | 16.33 MiB/s, done.
Resolving deltas: 100% (807/807), done.


Next, download the necessary dependencies.

In [6]:
!pip install -U pip
# if you are running this file on a Colab notebook or equivalent, run the line below. Otherwise, run the last line
!pip install -e ColBERT/['faiss-gpu','torch']
!pip install -e ColBERT/

Obtaining file:///content/ColBERT
  Preparing metadata (setup.py) ... done
  Attempting uninstall: colbert
    Found existing installation: ColBERT 0.2.0
    Can't uninstall 'ColBERT'. No files were found to uninstall.
  Running setup.py develop for colbert


In [7]:
import colbert

In [8]:
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

We will use the dev set of the HuggingFace **LoTTE benchmark** we recently introduced in the ColBERTv2 paper. The dev and test sets contain several domain-specific corpora, and we'll use the smallest dev set corpus, namely lifestyle:dev.

In [9]:
from datasets import load_dataset

dataset = 'lifestyle'
datasplit = 'dev'

collection_dataset = load_dataset("colbertv2/lotte_passages", dataset)
collection = [x['text'] for x in collection_dataset[datasplit + '_collection']]

queries_dataset = load_dataset("colbertv2/lotte", dataset)
queries = [x['query'] for x in queries_dataset['search_' + datasplit]]

f'Loaded {len(queries)} queries and {len(collection):,} passages'

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating dev_collection split: 0 examples [00:00, ? examples/s]

Generating test_collection split: 0 examples [00:00, ? examples/s]

Dataset lotte_passages downloaded and prepared to /root/.cache/huggingface/datasets/colbertv2___lotte_passages/lifestyle/1.1.0/8b0c6ee8d8641c804a3303248f83c490c72ec2df6c97cb48a9eb9caeea9d2fa0. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

...

Generating examples


Generating search_test split: 0 examples [00:00, ? examples/s]

Generating examples
Dataset lotte downloaded and prepared to /root/.cache/huggingface/datasets/colbertv2___lotte/lifestyle/1.1.0/1d2984a7d651f29dfe45f7023a0c3b76d44bdef8ccbebdfa3a9522fb5cdbc7df. Subsequent calls will reuse this data.


  0%|          | 0/4 [00:00<?, ?it/s]

'Loaded 417 queries and 268,881 passages'

This loaded 417 queries and 269k passages. Let's inspect one query and one passage to verify we have done so correctly.

In [10]:
print(queries[24])
print()
print(collection[19929])
print()

are blossom end rot tomatoes edible?

I think the spraying thing is not after, it's during. The cold will freeze the mist, keeping the air around the trees at (but not below) freezing. See http://www.ehow.com/how_5805520_use-freeze-damage-fruit-trees.html for example which recommends a sprinkler. The "releases heat" thing is kind of an oversimplification, but basically as long as you have any liquid water around, it will keep things at zero. The sap of your tree is not pure water, and therefore freezes somewhat below zero. By having the water freeze instead you stay away from the temps that would damage your plants. That said, http://www.ehow.com/how-does_5245655_spraying-frost-protect-fruit-freezing_.html is total gibberish since evaporation doesn't generate heat, quite the opposite. There is a better explanation at http://www.gardenguides.com/135830-spray-water-plants-during-frost.html This is a picture from a blog entry that gives you details from the citrus farmer's point of view.


## Indexing

For an efficient search, we can pre-compute the ColBERT representation of each passage and index them.

Below, the `Indexer` take a model checkpoint and writes a (compressed) index to disk. We then prepare a `Searcher` for retrieval from this index.

In [11]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300 # truncate passages at 300 tokens
max_id = 10000

index_name = f'{dataset}.{datasplit}.{nbits}bits'

To save space and time, we will only run the `Indexer` on the first 10,000 passages. To do so, we will filter out queries that do not contain passages with ids less than 10,000.

In [12]:
answers = [x['answers'] for x in queries_dataset['search_' + datasplit]]
answer_pids = [x['answer_pids'] for x in answers]
cur = 0 # tracks the current query index

while cur < len(queries):
  if any(x < max_id for x in answer_pids[cur]):
    cur += 1
  else:
    del queries[cur]

f'Filtered down to {len(queries)} queries'

'Filtered down to 20 queries'

Now run the `Indexer` on the collection subset. Assuming the use of only one GPU, this cell should take about seven minutes to finish running.

In [13]:
checkpoint = 'colbert-ir/colbertv2.0'

with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations performed on each doc

    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection[:max_id], overwrite=True)



[Jun 28, 20:30:41] #> Creating directory /content/experiments/notebook/indexes/lifestyle.dev.2bits 


#> Starting...
#> Joined...


In [14]:
indexer.get_index() # You can get the absolute path of the index, if needed.

'/content/experiments/notebook/indexes/lifestyle.dev.2bits'

## Search

Having built the index and prepared our `searcher`, we can search for individual query strings.

We can use the `queries` set we loaded earlier — or you can supply your own questions. Feel free to get creative! But keep in mind this set of ~300k lifestyle passages can only answer a small, focused set of questions!

In [15]:
# To create the searcher using its relative name (i.e., not a full path), set
# experiment=value_used_for_indexing in the RunConfig.
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name, collection=collection)


# If you want to customize the search latency--quality tradeoff, you can also supply a
# config=ColBERTConfig(ncells=.., centroid_score_threshold=.., ndocs=..) argument.
# The default settings with k <= 10 (1, 0.5, 256) gives the fastest search,
# but you can gain more extensive search by setting larger values of k or
# manually specifying more conservative ColBERTConfig settings (e.g. (4, 0.4, 4096)).

[Jun 28, 20:37:50] #> Loading codec...
[Jun 28, 20:37:50] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Jun 28, 20:37:50] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Jun 28, 20:37:50] #> Loading IVF...
[Jun 28, 20:37:50] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 973.83it/s]

[Jun 28, 20:37:50] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 14.65it/s]


In [23]:
query = queries[13] # try with an in-range query or supply your own
print(f"#> {query}")

# Find the top-3 passages for this query
results = searcher.search(query, k=3)

# Print out the top-k retrieved passages
for passage_id, passage_rank, passage_score in zip(*results):
    print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

#> are some cats just skinny?
	 [1] 		 24.1 		 A cat can certainly be naturally skinny. I know one who was the runt of her litter and has been extremely thin all her life, to the point where you can easily count every bone. She is now 17 years old, having outlived two other cats in that household, so it certainly doesn't seem to have held her back.
	 [2] 		 24.0 		 Yes. Just like us, cats vary in size and shape and weight. And like us, some of that is diet, some is health, some is genetics, some is age. One of my lady's cats was the runt of the litter; it wasn't certain she would survive, and she has always been both small and skinny. It hasn't seemed to limit her climbing/jumping much, if at all; I think she benefits from square/cube law to be stronger relative to her weight than you would expect. Another cat in the family was not only longer/taller/broader but also more solidly muscled. I think he may have weighed twice what the small one did, without being overweight. (The simplest 